In [0]:
pip install boto3 pandas sqlalchemy psycopg2-binary


Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install python-dotenv

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install gdown


Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%run ./env-setup

In [0]:
import pandas as pd
import boto3
import time
import os
from dotenv import load_dotenv
from io import StringIO
import requests
import gdown

In [0]:
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.environ.get("AWS_REGION")
S3_BUCKET = os.environ.get("S3_BUCKET")
S3_FOLDER = os.environ.get("S3_FOLDER")

In [0]:
# --- Step 1: Download CSVs from Google Drive --- #
# transactions_url = "https://drive.google.com/file/d/1AGXVlDhbMbhoGXDJG0IThnqz86Qy3hqb/view?usp=drive_link"
customer_importance_url = "https://drive.google.com/uc?export=download&id=1abe9EkM_uf2F2hjEkbhMBG9Mf2dFE4Wo"

print("Downloading files from Google Drive...")
try:
    # transactions = pd.read_csv(transactions_url, on_bad_lines='skip')  # Skip bad lines
    customer_importance = pd.read_csv(customer_importance_url)
except Exception as e:
    print(f"Error downloading files: {e}")
    raise

In [0]:
# Google Drive file ID
file_id = "1AGXVlDhbMbhoGXDJG0IThnqz86Qy3hqb"
output = "/tmp/transactions.csv"

gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

# Load into pandas
transactions = pd.read_csv(output)


Downloading...
From: https://drive.google.com/uc?id=1AGXVlDhbMbhoGXDJG0IThnqz86Qy3hqb
To: /tmp/transactions.csv
100%|██████████| 49.0M/49.0M [00:00<00:00, 72.8MB/s]


In [0]:
# Setup S3 client
s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

In [0]:
transactions.head(3)

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0


In [0]:
df=transactions.copy()
df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [0]:
# Stream transactions in chunks of 10,000 every second
CHUNK_SIZE = 10000
total_chunks = len(df) // CHUNK_SIZE + (1 if len(df) % CHUNK_SIZE else 0)

print(f"Streaming {total_chunks} chunks...")


Streaming 60 chunks...


In [0]:
# Stream transactions in chunks of 10,000 every second
CHUNK_SIZE = 10000
total_chunks = len(df) // CHUNK_SIZE + (1 if len(df) % CHUNK_SIZE else 0)

print(f"Streaming {total_chunks} chunks...")


for chunk_num in range(total_chunks):
    start_idx = chunk_num * CHUNK_SIZE
    end_idx = start_idx + CHUNK_SIZE
    chunk_df = df.iloc[start_idx:end_idx]

    filename = f"chunk_{chunk_num+1:04}.csv"
    local_path = f"/tmp/{filename}"
    s3_path = f"{S3_FOLDER}/{filename}"

    chunk_df.to_csv(local_path, index=False)
    
    # Upload to S3
    s3.upload_file(local_path, S3_BUCKET, s3_path)
    print(f"[{chunk_num+1}/{total_chunks}] Uploaded {filename} to s3://{S3_BUCKET}/{s3_path}")
    
    os.remove(local_path)
    # time.sleep(1)

Streaming 60 chunks...
[1/60] Uploaded chunk_0001.csv to s3://bank-transactions-vishak/input_chunks/chunk_0001.csv
[2/60] Uploaded chunk_0002.csv to s3://bank-transactions-vishak/input_chunks/chunk_0002.csv
[3/60] Uploaded chunk_0003.csv to s3://bank-transactions-vishak/input_chunks/chunk_0003.csv
[4/60] Uploaded chunk_0004.csv to s3://bank-transactions-vishak/input_chunks/chunk_0004.csv
[5/60] Uploaded chunk_0005.csv to s3://bank-transactions-vishak/input_chunks/chunk_0005.csv
[6/60] Uploaded chunk_0006.csv to s3://bank-transactions-vishak/input_chunks/chunk_0006.csv
[7/60] Uploaded chunk_0007.csv to s3://bank-transactions-vishak/input_chunks/chunk_0007.csv
[8/60] Uploaded chunk_0008.csv to s3://bank-transactions-vishak/input_chunks/chunk_0008.csv
[9/60] Uploaded chunk_0009.csv to s3://bank-transactions-vishak/input_chunks/chunk_0009.csv
[10/60] Uploaded chunk_0010.csv to s3://bank-transactions-vishak/input_chunks/chunk_0010.csv
[11/60] Uploaded chunk_0011.csv to s3://bank-transaction

In [0]:
%run ./Mechanism-Y

PostgreSQL tracking table ready

✅ All file processing complete.
Saved 74437 rows in 1489 batches of 50 rows each


In [0]:
test_file=spark.read.parquet(f's3a://{S3_BUCKET}/result/batch_id=1014/*.parquet')
test_file.show(3)

+--------------------+---------+----------+------------+-----------+
|       detectionTime|patternId|ActionType|customerName| MerchantId|
+--------------------+---------+----------+------------+-----------+
|2025-06-04 09:39:...|   PatId2|     CHILD|   C74457300|M1823072687|
|2025-06-04 09:39:...|   PatId2|     CHILD|  C274026379|M1823072687|
|2025-06-04 09:39:...|   PatId2|     CHILD|  C398651508|M1823072687|
+--------------------+---------+----------+------------+-----------+
only showing top 3 rows

